<a href="https://colab.research.google.com/github/kushv16/Audio-Sentiment-Analysis/blob/master/Cleaning_EMOV_DB_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
!pip install noisereduce

import pandas as pd
import numpy as np
import random
from glob import glob
import os
import librosa
import noisereduce as nr
from scipy.io import wavfile
import warnings
warnings.filterwarnings("ignore")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


First, let's import the dataset previously exported:

In [3]:
# Import the dataset and verify that the upload is successful
df = pd.read_csv('/content/drive/MyDrive/LY Project/Dataframes/Dataset/emov-db-df.csv', index_col=0)
print('The dataset has {} audio files'.format(df.shape[0]))
df.sample()

The dataset has 6893 audio files


,path,filename,dataset,duration,sample_rate,gender,age,emotion
4263,/kaggle/input/emovdb-sorted/josh/Sleepy/sleepi...,sleepiness_197-224_0207,EmoV_DB,3.825,16000,male,30,sleepy


In [8]:
paths = list(df['path'])

updated_paths = []
updated_filenames = []

for path in paths:
  path_components = path.split('/')
  updated_path_components = path_components[4:]
  path_string = "/"
  a = "/content/drive/MyDrive/Emov-db/" + path_string.join(updated_path_components)
  updated_paths.append(a)
  
  components = a.split('/')
  updated_filename = components[5] + "_" + components[-1]
  updated_filenames.append(updated_filename)
  
  

In [9]:
print(updated_paths[:10])
print(updated_filenames[:10])


['/content/drive/MyDrive/Emov-db/bea/Amused/amused_1-15_0001.wav', '/content/drive/MyDrive/Emov-db/bea/Amused/amused_1-15_0002.wav', '/content/drive/MyDrive/Emov-db/bea/Amused/amused_1-15_0003.wav', '/content/drive/MyDrive/Emov-db/bea/Amused/amused_1-15_0004.wav', '/content/drive/MyDrive/Emov-db/bea/Amused/amused_1-15_0005.wav', '/content/drive/MyDrive/Emov-db/bea/Amused/amused_1-15_0006.wav', '/content/drive/MyDrive/Emov-db/bea/Amused/amused_1-15_0007.wav', '/content/drive/MyDrive/Emov-db/bea/Amused/amused_1-15_0009.wav', '/content/drive/MyDrive/Emov-db/bea/Amused/amused_1-15_0010.wav', '/content/drive/MyDrive/Emov-db/bea/Amused/amused_1-15_0011.wav']
['bea_amused_1-15_0001.wav', 'bea_amused_1-15_0002.wav', 'bea_amused_1-15_0003.wav', 'bea_amused_1-15_0004.wav', 'bea_amused_1-15_0005.wav', 'bea_amused_1-15_0006.wav', 'bea_amused_1-15_0007.wav', 'bea_amused_1-15_0009.wav', 'bea_amused_1-15_0010.wav', 'bea_amused_1-15_0011.wav']


In [10]:
df['path'] = updated_paths
df['filename'] = updated_filenames
df.head(5)

,path,filename,dataset,duration,sample_rate,gender,age,emotion
0,/content/drive/MyDrive/Emov-db/bea/Amused/amus...,bea_amused_1-15_0001.wav,EmoV_DB,4.385,44100,female,30,amused
1,/content/drive/MyDrive/Emov-db/bea/Amused/amus...,bea_amused_1-15_0002.wav,EmoV_DB,4.944,44100,female,30,amused
2,/content/drive/MyDrive/Emov-db/bea/Amused/amus...,bea_amused_1-15_0003.wav,EmoV_DB,6.399,44100,female,30,amused
3,/content/drive/MyDrive/Emov-db/bea/Amused/amus...,bea_amused_1-15_0004.wav,EmoV_DB,4.058,44100,female,30,amused
4,/content/drive/MyDrive/Emov-db/bea/Amused/amus...,bea_amused_1-15_0005.wav,EmoV_DB,3.358,44100,female,30,amused


In [11]:
!pip install tqdm

from tqdm import tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [12]:
# Folder where to store the cleaned audio files
path = '/content/drive/MyDrive/LY Project/Cleaned Datasets/cleaned_samples_emodb/'

# Function to clean the samples based on dataset
def clean_files(dataset): # Select files linked to a dataset
    
    df_clean = df[df['dataset'] == dataset].reset_index(drop=True)
    
    for i in tqdm(range(0, df_clean.shape[0])):
        
        # load audio file at a sample rate of 16000 Hz:
        y, sr = librosa.load(df_clean.path[i], sr=16000)
    
        # Trim signal at a level of 20 db
        y_trim, _ = librosa.effects.trim(y, top_db=20)
        
        # Remove 25% noise from audio samples
        y_noise_rem = nr.reduce_noise(y=y_trim, sr=sr, prop_decrease=0.1, stationary=True)
        
        # Rename new file adding '_cleaned.wav' and put it in the new folder
        name = (os.path.join(path)+df_clean.filename[i]+'_cleaned.wav')
        #  Save output in a wav file in a new folder
        wavfile.write(name, sr, y_noise_rem)


In [13]:
%%time
clean_files('EmoV_DB')

100%|██████████| 6893/6893 [38:22<00:00,  2.99it/s]

CPU times: user 13min 8s, sys: 1min 16s, total: 14min 24s
Wall time: 38min 22s
